In [18]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [19]:
train_df = pd.read_csv('../data/raw/train.csv')
test_df = pd.read_csv('../data/raw/x_test.csv')

In [20]:
train_df.sku.unique()

array([2689, 1027, 2696, 2698, 1035,  144, 2704, 1554, 2705, 2707, 2711,
       2712, 1051, 2718,  546, 1058,  549, 1065,  554,  686,  688, 1206,
       2360, 2365, 1472, 1600, 1603, 1732, 1608, 2249, 1356, 1618, 1365,
       1371, 2396, 1633, 2401, 2410, 1516, 2678, 2681, 2682, 2683])

In [21]:
test_df.sku.unique()

array([1027, 1035,  144, 1051,  546, 1058,  549, 1065,  554,  686,  688,
       1206])

In [22]:
#drop 2689 becaouse it is not correlated to anything
train_df = train_df[train_df.sku != 2689]

In [23]:
train_df = train_df.rename({"Unnamed: 0":"date"}, axis=1)
train_df.date = train_df.date.apply(lambda x:datetime.strptime(x[3:], '%d %B %Y'))
test_df = test_df.rename({"Unnamed: 0":"date"}, axis=1)
test_df.date = test_df.date.apply(lambda x:datetime.strptime(x[3:], '%d %B %Y'))

In [24]:
# on train dataframe values of "2016-12-10" raw regarding the week -1 are null so we drop them
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5544 entries, 134 to 5718
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 5544 non-null   datetime64[ns]
 1   sku                  5544 non-null   int64         
 2   pack                 5544 non-null   object        
 3   size (GM)            5544 non-null   float64       
 4   brand                5544 non-null   object        
 5   price                5544 non-null   float64       
 6   POS_exposed w-1      5544 non-null   float64       
 7   volume_on_promo w-1  5544 non-null   float64       
 8   sales w-1            5544 non-null   float64       
 9   scope                5544 non-null   int64         
 10  target               5544 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(2)
memory usage: 519.8+ KB


Bisogna decidere se lasciare l'ultima riga del test con target nan (perchè andrà predetto per la consegna) o se togliere la riga (perchè non si riesce a calcolarci il mape non avendo il valore reale)

In [25]:
#make of target column on test set
temp = pd.DataFrame(columns = train_df.columns)

for sku in test_df['sku'].unique():
    sales_sku = test_df.loc[test_df['sku'] == sku]
    #nan on the last row becouse we don't have the value
    sales_sku['target'] = sales_sku["sales w-1"].shift(-1)
    temp = pd.concat((temp, sales_sku), axis = 0)

test_df = temp


In [26]:
quantile_transformer = preprocessing.QuantileTransformer(output_distribution='normal', random_state=123)
min_max_scaler = preprocessing.MinMaxScaler()

def scale(feature, scaler = min_max_scaler):
    size = len(feature)
    return scaler.fit_transform(np.array([feature]).reshape(size, 1)).T[0]
def unscale(scaled, original, scaler = min_max_scaler):
    size2 = len(scaled)
    size1 = len(original)
    return scaler.fit(np.array([original]).reshape(size1, 1)).inverse_transform(np.array([scaled]).reshape(size2, 1)).T[0]

In [27]:
def create_features(df):
    
    df2 = pd.DataFrame(columns= df.columns)
    sku_groups = {
        "A" :[1603, 2705, 1608, 1618, 1371],
        "B" :[2249, 2401, 2365, 2410],
        "C" :[1554, 1600],
        "D" :[2678, 2683],
        "E" :[1365, 1732, 1472, 2682],
        "F" :[144, 686, 1051],
        "G" :[688,1058,549,546,1027,1035,554,1206,1065,1365,1732,1472,1600,1554,1516,1633,1371
              ,1356,2678,2683,2681,1603,2705,1618,2365,2696,2718,2711,2712,2698,2396,2682,2707,2704,2360]         
    }
    

    for sku in df['sku'].unique():
        sales_sku = df.loc[df['sku'] == sku]
        
        for l,g in sku_groups.items():
            if sku in g:
                if "corr_group" in sales_sku.columns:
                    sales_sku["corr_group"] = sales_sku["corr_group"].apply(lambda x: x + " " + l)
                else:
                    sales_sku["corr_group"] = np.full((len(sales_sku["price"]),), l)
              
        sales_sku['sales w-2'] = sales_sku['sales w-1'].shift(1)
        sales_sku['sales w-3'] = sales_sku['sales w-1'].shift(2)
                    
        sales_sku["rolling1"] = sales_sku["sales w-1"].rolling(1).mean()
        sales_sku["rolling2"] = sales_sku["sales w-1"].rolling(2).mean()
        sales_sku["rolling3"] = sales_sku["sales w-1"].rolling(3).mean()
        sales_sku["rolling4"] = sales_sku["sales w-1"].rolling(4).mean()
        sales_sku["rolling5"] = sales_sku["sales w-1"].rolling(5).mean()

        sales_sku['diff1'] = -(sales_sku['sales w-1'] - sales_sku['sales w-1'].shift(1))
        sales_sku['diff2'] = -(sales_sku['sales w-1'] - sales_sku['sales w-1'].shift(2))
        sales_sku['diff3'] = -(sales_sku['sales w-1'] - sales_sku['sales w-1'].shift(3))
 
        sales_sku["scaled_target"] = scale(sales_sku["target"])
        sales_sku["scaled_price"] = scale(sales_sku["price"])
        sales_sku['scaled_sales1'] = scale(sales_sku['sales w-1'])
        sales_sku["scaled_promo"] = scale(sales_sku["volume_on_promo w-1"])
        
        sales_sku["scaled_rolling1"] = sales_sku["scaled_sales1"].rolling(1).mean()
        sales_sku["scaled_rolling2"] = sales_sku["scaled_sales1"].rolling(2).mean()
        sales_sku["scaled_rolling3"] = sales_sku["scaled_sales1"].rolling(3).mean()
        sales_sku["scaled_rolling4"] = sales_sku["scaled_sales1"].rolling(4).mean()
        sales_sku["scaled_rolling5"] = sales_sku["scaled_sales1"].rolling(5).mean()
        
        sales_sku['scaled_diff1'] = -(sales_sku['scaled_sales1'] - sales_sku['scaled_sales1'].shift(1))
        sales_sku['scaled_diff2'] = -(sales_sku['scaled_sales1'] - sales_sku['scaled_sales1'].shift(2))
        sales_sku['scaled_diff3'] = -(sales_sku['scaled_sales1'] - sales_sku['scaled_sales1'].shift(3))
        sales_sku['percentage_diff1'] = -(sales_sku['sales w-1'] - sales_sku['sales w-1'].shift(1))/sales_sku['sales w-1']
        sales_sku['scaled_price_diff1'] = -(sales_sku['scaled_price'] - sales_sku['scaled_price'].shift(1))
      
        df2 = pd.concat((df2, sales_sku), axis = 0)
    
    #df2 = df2.dropna()
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = df2["corr_group"].str.get_dummies(" ") > 0
    onehot = pd.DataFrame(onehot_encoded, index = df2.index)
    
    onehot_encoded = df2["brand"].str.get_dummies() > 0
    onehot2 = pd.DataFrame(onehot_encoded, index = df2.index)

    df = pd.concat([df2, onehot, onehot2], axis=1, sort=False)
    
    df = df.drop(columns=["pack","size (GM)","brand","corr_group","POS_exposed w-1"])
    
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    df['scaled_quarter'] = scale(df['quarter'])
    df['scaled_month'] = scale(df['month'])
    df['scaled_year'] = scale(df['year'])
    df['scaled_dayofyear'] = scale(df['dayofyear'])
    df['scaled_dayofmonth'] = scale(df['dayofmonth'])
    df['scaled_weekofyear'] = scale(df['weekofyear'])
    
    df["sku"] = df["sku"].apply(lambda x: int(x))
    
    cols = list(df.columns.values)
    cols.pop(cols.index('target'))
    cols.pop(cols.index('scaled_target'))
    df = df[cols+['scaled_target','target']]
    return df

In [28]:
train_df["scope"] = train_df.scope.apply(lambda x: int(x))

In [29]:
train_df_scope0 = train_df.loc[train_df.scope == 0]
train_df_scope1 = train_df.loc[train_df.scope == 1]

In [30]:
all_df1 = pd.concat([train_df_scope1,test_df])

In [31]:
processed_train_scope0 = create_features(train_df_scope0)

In [32]:
all_df1 = create_features(all_df1)
all_df1 = all_df1.set_index("date")
processed_train_scope1 = all_df1[:"2019-06-22"].reset_index()
processed_test = all_df1["2019-06-23":].reset_index()

In [40]:
processed_train = pd.concat([processed_train_scope0,processed_train_scope1])
processed_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5544 entries, 267 to 1583
Data columns (total 55 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 5544 non-null   datetime64[ns]
 1   sku                  5544 non-null   int64         
 2   price                5544 non-null   float64       
 3   volume_on_promo w-1  5544 non-null   float64       
 4   sales w-1            5544 non-null   float64       
 5   scope                5544 non-null   object        
 6   sales w-2            5502 non-null   float64       
 7   sales w-3            5460 non-null   float64       
 8   rolling1             5544 non-null   float64       
 9   rolling2             5502 non-null   float64       
 10  rolling3             5460 non-null   float64       
 11  rolling4             5418 non-null   float64       
 12  rolling5             5376 non-null   float64       
 13  diff1                5502 non-n

In [41]:
processed_train["BRAND1"] = processed_train["BRAND1"].fillna(False)
processed_train["BRAND2"] = processed_train["BRAND2"].fillna(False)
processed_train["BRAND3"] = processed_train["BRAND3"].fillna(False)
processed_train["BRAND4"] = processed_train["BRAND4"].fillna(False)
processed_train["BRAND5"] = processed_train["BRAND5"].fillna(False)
processed_train["A"] = processed_train["A"].fillna(False)
processed_train["B"] = processed_train["B"].fillna(False)
processed_train["C"] = processed_train["C"].fillna(False)
processed_train["D"] = processed_train["D"].fillna(False)
processed_train["E"] = processed_train["E"].fillna(False)
processed_train["F"] = processed_train["F"].fillna(False)
processed_train["G"] = processed_train["G"].fillna(False)

In [42]:
processed_train.to_csv("processed_train.csv")
processed_test.to_csv("processed_test.csv")